In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install neo4j
%pip install json-repair

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from neo4j import GraphDatabase

from dotenv import load_dotenv

load_dotenv()

/Users/larissakemicloud.com/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [9]:
NEO4J_URI = os.environ["NEO4J_URI"] 
NEO4J_USERNAME = os.environ["NEO4J_USERNAME"]
NEO4J_PASSWORD =os.environ["NEO4J_PASSWORD"]

# Criar driver de conexão
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Exemplo de comando no banco de dados 'system'
def create_database():
    with driver.session(database="system") as session:
        session.run("CREATE DATABASE meu_novo_database")

# Criar um novo banco de dados
create_database()

ClientError: {code: Neo.ClientError.Statement.UnsupportedAdministrationCommand} {message: Unsupported administration command: CREATE DATABASE meu_novo_database}

In [ ]:
# Agora você pode criar o gráfico sem passar os parâmetros diretamente
graph = Neo4jGraph()

In [8]:
# Carregar os conteúdos dos arquivos SQL fornecidos
file_paths = [
    '/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupo.sql',
    '/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupoUsuario.sql',
]

# Carregar os documentos
docs = []
for file_path in file_paths:
    loader = TextLoader(file_path, encoding="utf-8")
    docs.extend(loader.load())  # Adicionar documentos carregados à lista

# Dividir os documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

# Verificar os documentos divididos
print(f"Total de documentos divididos: {len(documents)}")

Total de documentos divididos: 6


In [9]:
documents[0]

Document(metadata={'source': '/Users/larissakemicloud.com/Documents/GitHub/GraphRAG-with-Llama-3.1/tables/ControleDeAcessoGrupo.sql'}, page_content='\ufeffCREATE TABLE [dbo].[ControleDeAcessoGrupo] (\n    [ID]                      INT            IDENTITY (1, 1) NOT NULL,\n    [ID_Empresa]              INT            NOT NULL,\n    [Nome]                    NVARCHAR (250) NOT NULL,')

In [10]:
# Obter as chaves da API
openai_api_key = os.getenv("OPENAI_API_KEY")
llama_api_key = os.getenv("LLAMA_API_KEY")

# Definir qual LLM será utilizado
llm_type = os.getenv("LLM_TYPE", "openai")  # Valor padrão "openai" se não estiver definido

if llm_type == "ollama":
    # Usar o modelo LLaMA/Ollama e passar a chave de API da LLaMA
    llm = ChatOllama(model="llama3.1", temperature=0, api_key=llama_api_key)
else:
    # Usar o modelo OpenAI e passar a chave de API da OpenAI
    llm = ChatOpenAI(temperature=0, model="gpt-4o", api_key=openai_api_key)

# Usar o modelo LLM no transformador de grafos
llm_transformer = LLMGraphTransformer(llm=llm)

# Converter documentos para o formato de grafo
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [11]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)